In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from collections import Counter
from datetime import datetime


In [9]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [10]:
sqlContext = SQLContext(sc)

In [4]:
logs = sqlContext.read.csv('logs.csv', header=True, inferSchema=True)

In [5]:
logs_rdd = logs.rdd.filter(lambda x: x[0] != None and x[0].isnumeric())

In [6]:
logs_usuarios_bloqueados = logs_rdd.filter(lambda x: x[7] == "block" and x[8] != None).map(lambda x: (x[8].replace("Usuario:", ""), datetime.strptime(x[1], '%Y-%m-%dT%H:%M:%SZ')))

In [7]:
logs_usuarios_creados = logs_rdd.filter(lambda x: x[7] == "newusers" and x[8] != None).map(lambda x: (x[2], datetime.strptime(x[1], '%Y-%m-%dT%H:%M:%SZ')))

In [11]:
logs_usuarios_bloqueados.join(logs_usuarios_creados).filter(lambda x: x[1][1]<x[1][0]).map(lambda x: (x[0] ,x[1][0]-x[1][1])).reduce(lambda x,y: x if x[1] < y[1] else y)

('GhostBar', datetime.timedelta(seconds=6))